In [3]:
import re

output_file = "overlap_results_SNP.tsv"
fout = open(output_file, 'w')

class Interval:

    def __init__(self, chrom, start, end, feature, gene_id=None, gene_name=None):
        '''
        Constructor, containing features representing an area on the chromosome.
        chrom: chromosome
        start: Starting position of the area on chr.
        end: Ending position of the area on chr.
        feature: Type of genetic feature represented by this area (e.g., "gene", "exon").
        gene_id: Gene identifier.
        gene_name: Gene name.
        '''
        self.chrom = chrom
        self.start = start
        self.end = end
        self.feature = feature
        self.gene_id = gene_id
        self.gene_name = gene_name

    def overlaps(self, other):
        # Does this area on the chromosome overlap with another area on the chromosome?
        return self.chrom == other.chrom and self.end >= other.start and self.start <= other.end

    def distance_to(self, other):
        # Calculate the distance between this area on the chromosome and another area on the chromosome.
        # If areas on the chromosome overlap, the distance between them is 0.
        if self.chrom != other.chrom:
            return float('inf')
        if self.end < other.start:
            return other.start - self.end
        elif other.end < self.start:
            return self.start - other.end
        else:
            return 0

def extract_gene_info(attributes):
    # Extract gene identifier and name information from attributes.
    gene_id_match = re.search('gene_id "([^"]+)"', attributes)
    gene_name_match = re.search('gene_name "([^"]+)"', attributes)

    gene_id = gene_id_match.group(1) if gene_id_match else None
    gene_name = gene_name_match.group(1) if gene_name_match else None

    return gene_id, gene_name

# Load the GTF file with the GRCh38 version of the genome
gtf_file = "Homo_sapiens.GRCh38.110.gtf"

# Dictionary of genetic features
features_dict = {
    "exon": [],
    "transcript": [],
    "CDS": [],
    "start_codon": [],
    "stop_codon": [],
    "three_prime_utr": [],
    "five_prime_utr": [],
    "gene": []    
}
coding_genes = set()


with open(gtf_file, 'r') as fin:
    # Open GTF file
    for line in fin:
        if not line.startswith('#'):  # Skip comments
            fields = line.strip().split('\t')
            if len(fields) != 9:
                print(f"Unexpected line format: {line}")
                continue

            # Assign attributes
            chrom = fields[0]
            start = int(fields[3]) - 1
            end = int(fields[4])
            feature = fields[2]
            gene_id, gene_name = extract_gene_info(fields[8])

            if feature == "CDS":
                # Is the processed genetic feature "CDS"?
                coding_genes.add(gene_id)

            if feature in features_dict:
                # Does the processed genetic feature correspond to one in the dictionary?
                features_dict[feature].append(Interval(chrom, start, end, feature, gene_id, gene_name))

input_file = "output_results_SNP.tsv"

with open(input_file, 'r') as fin:
    # Open file with rsIDs and coordinates
    headers = next(fin).strip().split('\t')
    rsid_index = headers.index('rsID')
    location_index = headers.index('Location')

    # For each rsID, find genetic features it overlaps with
    for line in fin:
        fields = line.strip().split('\t')
        # get rsID and coordinates from input file
        rsid = fields[rsid_index]
        location = fields[location_index]
        chrom, pos = location.split(':')
        start, end = map(int, pos.split('-'))

        # Create Interval object for rsID coordinates
        coord_interval = Interval(chrom, start, end, "Query")
        
        overlapping_features = set()
        
        # For each genetic feature in the dictionary of genetic features, find features overlapping with rsID
        for feature, intervals in features_dict.items():
        #Iterate through the dictionary of genetic features
            for interval in intervals:
                if interval.overlaps(coord_interval):
                #If features overlap,, add them to the set of features overlapping with rsID
                    if interval.gene_name:
                    #If gene name is known, add it to the set of features overlapping with rsID
                        overlapping_features.add(f"{feature}_{interval.gene_name}")
                    elif interval.gene_id:
                    #If gene name is not known, but its identifier is known, add it to the set of features overlapping with rsID
                        overlapping_features.add(f"{feature}_{interval.gene_id}")
                    else:
                        overlapping_features.add(feature)
                        
        nearest_gene_distance = float('inf')
        nearest_gene_info = None
        for gene in features_dict["gene"]:
            #Iterate through the list of coding genes
            if gene.gene_id in coding_genes:
                #Is the gene coding?
                distance = coord_interval.distance_to(gene)
                #Calculate the distance between rsID and the gene
                if distance < nearest_gene_distance:
                    #If the distance is smaller than the smallest distance, update the information about the nearest gene
                    nearest_gene_distance = distance
                    #If the gene name is known, add it to the set of features overlapping with rsID
                    nearest_gene_info = gene.gene_name if gene.gene_name else gene.gene_id

        # Write to file rsID, features overlapping with rsID, nearest coding gene
        overlap_str = ', '.join(overlapping_features) if overlapping_features else "No direct overlap"
        nearest_gene_str = nearest_gene_info if nearest_gene_info else "No nearby coding gene found"
        fout.write(f"{rsid} overlaps with: {overlap_str}. Nearest coding gene: {nearest_gene_str}\n")

fout.close()
